<a href="https://colab.research.google.com/github/thdgg/Emotion-Detection-for-Text-based-Communication/blob/main/XLNet_Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [2]:
!pip install accelerate -U

In [3]:
!pip install 'transformers[torch]'

In [4]:
import tensorflow as tf
from datasets import load_dataset
from transformers import XLNetTokenizer, XLNetForSequenceClassification, TrainingArguments, Trainer

In [5]:
dataset = load_dataset("dair-ai/emotion")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

In [6]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=300)


processed_dataset = dataset.map(preprocess_function, batched=True)

In [7]:
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=6)  # 6 emotion labels

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy='steps'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"]
)

trainer.train()

Step,Training Loss,Validation Loss
500,0.937200,0.389676
1000,0.343900,0.234210
1500,0.210300,0.244885
2000,0.196700,0.204416
2500,0.140400,0.191220
3000,0.119500,0.198996


TrainOutput(global_step=3000, training_loss=0.3246735051472982, metrics={'train_runtime': 4987.4653, 'train_samples_per_second': 9.624, 'train_steps_per_second': 0.602, 'total_flos': 8012523628800000.0, 'train_loss': 0.3246735051472982, 'epoch': 3.0})

In [32]:
def detect_emotion(text):
    inputs = tokenizer(text, return_tensors="pt")
    inputs = inputs.to('cuda')
    outputs = model(**inputs)
    predicted_class_id = outputs.logits.argmax().item()
    return dataset['train'].features['label'].names[predicted_class_id]


text_sample = "Today, the sky is blue"
emotion = detect_emotion(text_sample)
print(f"Detected Emotion: {emotion}")

Detected Emotion: joy
